# Julia nivel intermedio

## Definición de nuevas estructuras (o tipos)

Ahora veremos cómo crear nuevas estructuras o tipos. Esto es algo impresionantemente útil, ya que permite definir un arreglo compacto de datos de interés y crear funciones especializadas. Dado que Julia se basa en el concepto de tipo, esto se traduce en velocidad.

Como ilustración definiremos un tipo nuevo, `Vector2D`, que permite dar la posición, o velocidad, de una partícula *libre*.

**NOTA** En Julia **no** se permite sobreescribir los tipos; es por esto que en varias ocasiones usaremos el comando `workspace()`, que esencialmente reseatea lo que se ha hecho en la sesión.

### Los campos de un `type`

Para entender un poco más lo que significa definir un nuevo tipo, consideremos los números complejos:

In [ ]:
z0 = complex(3,2) # z=3+2im

In [ ]:
typeof(ans)

La instrucción `fieldnames` da los campos internos del tipo de la variable `z0`; vale la pena notar que el resultado consiste en un vector de *símbolos*. Estos símbolos son los nombres de los campos internos del tipo.

In [ ]:
fieldnames(z0)

Uno puede acceder el valor de los campos a través de `z0.re` o `z0.im`:

In [ ]:
z0.re, z0.im

In [ ]:
typeof(ans)

Consideremos otro ejemplo, los números racionales:

In [ ]:
BigInt(1)//2

In [ ]:
typeof(ans)

In [ ]:
fieldnames(BigInt(1)//2)

### Vectores de tamaño fijo `Vector2D`

La sintáxis básica para definir nuevos tipos es:

In [ ]:
type Vector2D
    x :: Float64
    y :: Float64
end

En la celda anterior, los doble dos-puntos `::` sirve para *anotar* el tipo de las variables internas. Esto es, `Vector2D` constará de dos componentes, `x` y `y`, ambas del tipo `Float64`.

Uno puede **no** especificar el tipo de las variables internas de la estructura. Esto puede ser útil en ciertas ocasiones, pero en general hace que el compilador no sepa qué tipo de variable son los campos internos y por lo tanto tiene que guardar la información con apuntadores. En otras palabras, esto puede generar código lento.

Por **convención**, las estructuras de tipo empiezan con una *mayúscula*; las funciones empiezan en *minúsculas*.

A estas alturas del partido, sólo hemos definido el tipo `Vector2D`; ¿hemos acaso definido algún método asociado a ésto? O, en otras palabras, ¿cómo creamos algo que sea del tipo `Vector2D`?

Veamos cuántos métodos hemos definido en torno a `Vector2D`:

In [ ]:
methods(Vector2D)

Claramente, hemos definido tres métodos simplemente por haber construido la estructura `Vector2D`. Vale la pena notar que el tercer método es intrínseco en Julia.

In [ ]:
(Vector2D)(1.0, 2.0) # Ejemplo del uso del tercer método

In [ ]:
Vector2D((1, 2)...) # "..." es el 'splat' operator

Los puntos suspensivos `...` implican que la *tupla* (1,2) será separada en sus componentes individuales, cada una considerada como un parámetro independiente.

In [ ]:
x = Vector2D(1, 4)

In [ ]:
y = Vector2D(5, 6)

In [ ]:
fieldnames(Vector2D)

In [ ]:
x.x

Vale la pena notar que uno puede modificar los campos internos de un tipo:

In [ ]:
x.y = 3

In [ ]:
x

**NOTA:** Uno también puede definir estructuras en que **no** sea posible modificar los valores internos; un ejemplo es `Rational`; a este último tipo de estructuras se les llama inmutables, y se crean usando `immutable` en lugar de `type`.

Sin importar mucho el significado de cada componente, la pregunta es si podemos sumar dos de ellos. Problemos:

In [ ]:
x + y

Julia nos arroja un error ya que *no* está definida la función `+` para dos argumentos del tipo `Vector2D`; el mensaje de la primer línea es perfectamente claro.

Entonces, dado que *no* está definido, y nos interesa usarlo, podemos simplemente definirlo. Entonces, como `+` existe como función, primero debemos importarlo (del módulo `Base`) y luego extender su definición.

In [ ]:
import Base.+
+(a::Vector2D, b::Vector2D) = Vector2D( a.x+b.x, a.y+b.y )

In [ ]:
x + y

De igual manera procedemos con las demás operaciones.

In [ ]:
import Base.-

-(a::Vector2D, b::Vector2D) = Vector2D(a.x-b.x, a.y-b.y)

In [ ]:
x - y

In [ ]:
import Base.*
*(a::Vector2D, b::Vector2D) = Vector2D(a.x*b.x, a.y*b.y)

In [ ]:
x * y

Vale la pena hacer notar que esto se puede usar en funciones directamente, siempre y cuando las operaciones o funciones involucradas tengan sentido.

In [ ]:
g(x,y) = x^2 - y^3

In [ ]:
x^2

In [ ]:
x^3


In [ ]:
g(x,y)

Para saber qué método concreto se utilizó, podemos usar el macro `@which`:

In [ ]:
@which x + y

In [ ]:
@which x^2

### Tipos parametrizados

Hay situaciones en que nos interesa definir un nuevo tipo para distintos tipos de signatura de los parámetros internos. Un ejemplo es `Complex` o `Rational`, pero de hecho también los arreglos son estructuras que reconocen el tipo de los elementos internos que los componen

In [ ]:
typeof(complex(1.0,2.0))

In [ ]:
typeof(1//2)

In [ ]:
typeof( [1, 2.1] )

El tipo `Vector2D` que definimos arriba funciona, pero es rígido en el sentido de que sus campos son *siempre* `Float64`. Por ejemplo:

In [ ]:
Vector2D(1//2, 1//1) + Vector2D(1//3, 1//1)

In [ ]:
Vector2D(complex(1,2), complex(1,1))

La razón por la que funcionó la instrucción penúltima es que Julia incluye métodos de conversión y promoción, que son funciones y que uno puede ampliar. Esto es, los racionales pueden ser *promovidos* a `Float64`; sin embargo, el error de la última instrucción muestra que los complejos **no** pueden ser promovidos a `Float64`.

Noten arriba que el entero "1" es *promovido* a Float64 "1.0", que es el tipo del segundo elemento.

Para *redefinir* el tipo `Vector2D` tenemos que borrar los tipos definidos (y todo lo demás) en el llamado ambiente global (*global scope*); esto lo hacemos con `workspace()`:

In [ ]:
workspace()

In [ ]:
x

In [ ]:
immutable Vector2D{T}
    x::T
    y::T
end

In [ ]:
methods(Vector2D)

En la celda anterior, hemos usado `immutable` en lugar de `type`, lo que le permite al compilador guardar (en memoria) a este tipo de estructura de una manera más eficiente. Esto, sin embargo, tiene consecuencias, como es el hecho de que una vez definida una estructura, sus componentes *no* pueden cambiar.

En la definición de `Vector2D{T}`, `T` representa el tipo de los elementos de la  estructura que creamos y *parametriza* a `Vector2D` lo que se indica con `{T}`. Dada esta definición, ambas componentes `x` y `y` tienen que ser del mismo tipo `T`.

In [ ]:
x = Vector2D(1,3)

y = Vector2D(2.0,3.5)

println(x)

In [ ]:
x.x

In [ ]:
x.x = 5

In [ ]:
x

Claramente **no** podemos cambiar el valor de las componentes internas de la estructura.

El definir una estructura parametrizada hace que, por default, si los parámetros **no** cumplen la signatura del tipo, haya un error. En este caso, las reglas de promoción no aplican; amboas componentes deben ser del mismo tipo `T`.

In [ ]:
Vector2D(1, 2.3)

Noten que la actual definición de `Vector2D{T}` (parametrizada) es *demasiado* flexible, esto es, permite definir ciertos tipos que, quizás, no nos interesa considerar. Por ejemplo:

In [ ]:
Vector2D("perro", "gato")

Vale la pena mencionar que las funciones, de hecho, también pueden ser parametrizadas:

In [ ]:
import Base.+

+{T}(a::Vector2D{T}, b::Vector2D{T}) = Vector2D{T}(a.x+b.x, a.y+b.y)

Noten arriba que la función, de hecho, está parametrizada, y que el parámetro está relacionado con el parámetros del tipo de los argumentos. Esta función se usará (y definirá un método especializado) cuando *ambos* argumentos son del tipo `Vector2D{T}`.

Si bien el valor de los argumentos *no* se conoce a la hora de compilar, el tipo sí.



In [ ]:
Vector2D(1,3) + Vector2D(-1,2)

In [ ]:
Vector2D(1,3) + Vector2D(-1//2,2//2)

El error es claro: no hemos definido cómo sumar `Vector2D{Int64}` con `Vector2D{Rational{Int64}}`; únicamente hemos definido la suma de dos `Vector2D{Int64}` o de dos `Vector2D{Rational{Int64}}`, pero ambos deben ser del mismo tipo, incluyendo sus parametrizaciones.

Uno, de hecho, podría haber definido la misma operación sin incluir la parte paramétrica; esto tiene consecuencias interesantes, que de hecho, a veces son deseables. Así, tenemos:

In [ ]:
import Base.-
-(a::Vector2D, b::Vector2D) = Vector2D(a.x-b.x, a.y-b.y)

In [ ]:
Vector2D(1,3) - Vector2D(-1,2)

In [ ]:
Vector2D(1//1,3//7) - Vector2D(3.0,1.1)

In [ ]:
Vector2D(1,3) - Vector2D("Soy", "Luis")

Gracias a que *no* impusimos que `a` y `b` sean *ambos* del tipo `Vector2D{T}`, Julia puede restarlos.

### Promotion and convertion

La pregunta es, cómo conseguir que *no* nos de un error la instrucción `Vector2D(1, 3.2)`. La respuesta tiene que ver con definir reglas de conversión y promoción. (En este caso concreto, como veremos abajo, una regla de promoción es suficiente.)

Convertir entre tipos (donde esto tenga sentido) se logra usando `convert`:

In [ ]:
convert(Float64, 1//2)

In [ ]:
convert(Array{Float64,1}, [1//2, 1//3])

In [ ]:
x = convert(Rational{Int128}, 0.5)

In [ ]:
typeof(x)

La operación de conversión debe tener sentido; por ejemplo, ¿qué esperamos que nos de `convert(Int64, 0.6)`?

In [ ]:
convert(Int64, 0.6)

También, existe la promoción de varios objetos a un tipo común; el resultado de `promotion` es una *tupla* de elementos promovidos a un tipo común.

In [ ]:
tup = promote(1, 1//2, BigInt(2))

In [ ]:
typeof(tup)

Las reglas de promoción siguen el árbol jerárquico en la organización de los tipos. El punto importante es que, tanto `convert` como `prmotion` son funciones y uno puede añadirles nuevos métodos según convenga.

In [ ]:
?Int

In [ ]:
?Signed

Lo anterior muestra que hay ciertos tipos *concretos* (como `Int64`, `Float64`) y *otros* que son abstractos; los segundos en algún sentido agrupan varios tipos distintos.

La instrucción `supertype` permite ver qué tipo está directamente arriba en la estructura del árbol:

In [ ]:
supertype(Float64)

In [ ]:
supertype(AbstractFloat)

In [ ]:
supertype(Real)

... y la instrucción subtypes, qué está por debajo:

In [ ]:
subtypes(Real)

In [ ]:
subtypes(Number)

La estructura de árbol tiene un tope *por arriba*, `Any`:

In [ ]:
super(Number)

In [ ]:
super(Any)

In [ ]:
subtypes(Any)

Vale la pena notar que existe, entre varias cosas, el tipo `Void` (antes `Nothing`)

In [ ]:
? Void

Volviendo al punto de cómo hacer para que `Vector2D{1, 3.2}` funcione, podemos definir un nuevo *método*, para la creación del tipo `Vector2D`, que involucre una promoción de tipos, de la siguiente manera:

In [ ]:
Vector2D(a, b) = Vector2D(promote(a,b)...)

In [ ]:
methods(Vector2D)

In [ ]:
Vector2D(1, 3.2)

Obviamente, esto sólo tendrá sentido cuando la promoción tiene sentido; cuando no la tiene, arrojará un error:

In [ ]:
Vector2D("Hola", 3.2)

Por último, uno puede restringir el tipo de parámetros de una estructura, usando la notación ` T <: R` que significa que el parámetro `T` es subtipo de `R`. Un ejemplo es `T <: Real`.

In [ ]:
workspace()

In [ ]:
type Vector2D{T<:Real}
    x :: T
    y :: T
end

Agregamos la promoción de los tipos:

In [ ]:
Vector2D(a, b) = Vector2D(promote(a,b)...)

In [ ]:
Vector2D(1, 3.2)

In [ ]:
Vector2D("Soy", "Luis")

La notación `T <: R` de hecho puede ser utilizada para saber si un tipo es subtipo de otro:

In [ ]:
Int64 <: Real

In [ ]:
Int64 <: Void

In [ ]:
Void <: Int64

Las dos últimas instrucciones muestran que `Void` e `Int64` pertenecen a dos ramas distintas del árbol jerárquico de tipos; en otras palabras, el tipo superior común entre `Int64` y `Void` es `Any`:

In [ ]:
promote_type(Int64, Void)

Finalmente, para definir que algo una estructura específica es un subtipo de algo específico, usamos `<:` nuevamente:

In [ ]:
workspace()

In [ ]:
type Vector2D{T<:Real} <: Real
    x :: T
    y :: T
end

In [ ]:
Vector2D(a, b) = Vector2D(promote(a,b)...)

In [ ]:
Vector2D(3, 1)

Agregamos otro constructor externo:

In [ ]:
Vector2D{T<:Real}(a::T) = Vector2D(a, a)

In [ ]:
Vector2D(2.1)

Lo siguiente sirve para evitar muchas definiciones...

In [ ]:
import Base: convert

convert(::Type{Vector2D}, b::Real) = Vector2D(b)

convert{T<:Real, S<:Real}(::Type{Vector2D{T}}, b::S) = Vector2D(convert(T,b))

convert{T<:Real}(::Type{Vector2D{T}}, b::T) = Vector2D(b)

convert{T<:Real, S<:Real}(::Type{Vector2D{T}}, b::Vector2D{S}) = Vector2D(convert(T,b.x), convert(T,b.y))


In [ ]:
convert(Vector2D, 1.2)

In [ ]:
convert(Vector2D{Float64},1)

In [ ]:
import Base: promote_rule, promote

promote_rule{T<:Real}(::Type{Vector2D{T}}, ::Type{T}) = Vector2D{T}
promote_rule{T<:Real, S<:Real}(::Type{Vector2D{T}}, ::Type{S}) = Vector2D{promote_type(T, S)}

In [ ]:
promote_type(Float64,Vector2D{Float64})

Extendemos las operaciones aritméticas

In [ ]:
import Base: +, -, *, /, ^

+(a::Vector2D, b::Vector2D) = Vector2D(a.x+b.x, a.y+b.y)

-(a::Vector2D, b::Vector2D) = Vector2D(a.x-b.x, a.y-b.y)

*(a::Vector2D, b::Vector2D) = Vector2D(a.x*b.x, a.y*b.y)



In [ ]:
Vector2D(2,1) + 2